# VERSPM Model Interface

In [1]:
import emat
import os
import pandas as pd
import numpy as np
import gzip
from emat.util.show_dir import show_dir, show_file_contents

This notebook is meant to illustrate the use of TMIP-EMAT's
various modes of operation.  It provides an illustration of how to use 
TMIP-EMAT and the demo interface to run the command line version
of the [Road Test](https://tmip-emat.github.io/source/emat.examples/RoadTest/road_test_yaml.html) 
model. A similar approach can be developed to run
any transportation model that can be run from the command line, including
for proprietary modeling tools that are typically run from a graphical
user interface (GUI) but that provide command line access also.

In this example notebook, we will activate some logging features.  The 
same logging utility is written directly into the EMAT and the
`core_files_demo.py` module. This will give us a view of what's happening
inside the code as it runs.

In [2]:
import logging
from emat.util.loggers import log_to_stderr
log = log_to_stderr(logging.INFO)

## Connecting to the Model

The interface for this model is located in the `core_files_demo.py`
module, which we will import into this notebook.  This file is extensively
documented in comments, and is a great starting point for new users
who want to write an interface for a new bespoke travel demand model.

In [3]:
import emat_verspm

Within this module, you will find a definition for the 
`RoadTestFileModel` class.

We initialize an instance of the model interface object.
If you look at the module code, you'll note the `__init__` function
does a number of things, including creating a temporary directory
to work in, copying the needed files into this temporary directory,
loading the scope, and creating a SQLite database to work within.
For your implementation, you might or might not do any of these steps.
In particular, you'll probably want to use a database that is
not in a temporary location, so that the results will be available
after this notebook is closed.

In [4]:
fx = emat_verspm.VERSPModel()

[00:03.26] MainProcess/WARNING: changing cwd to /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmp2v_r9ty1
[00:03.27] MainProcess/INFO: running script scope.sql
[00:03.28] MainProcess/INFO: running script exp_design.sql
[00:03.28] MainProcess/INFO: running script meta_model.sql


Once we have loaded the `RoadTestFileModel` class, we have
a number of files available in the "master_directory" that 
was created as that temporary directory:

## Understanding Directories

The TMIP-EMAT interface design for files-based bespoke models uses
pointers for several directories to control the operation of the 
model.

- **local_directory**  
    This is the working directory for this instance of TMIP-EMAT,
    not that for the core model itself. Typically it can be Python's 
    usual current working directory, accessible via `os.getcwd()`.
    In this directory typically you'll have a TMIP-EMAT model 
    configuration *yaml* file, a scope definition *yaml* file, and
    a sub-directory containing the files needed to run the core model
    itself.

- **model_path**  
    The relative path from the `local_directory` to the directory where
    the core model files are located.  When the core model itself is actually
    run, this should be to the "current working directory" for that run.
    The `model_path` must be given in the model config *yaml* file.

- **rel_output_path**  
    The relative path from the `model_path` to the directory where
    the core model output files are located. The default value of this 
    path is "./Outputs" but this can be overridden by setting 
    `rel_output_path` in the model config *yaml* file. If the outputs
    are comingled with other input files in the core model directory,
    this can be set to "." (just a dot).

- **archive_path**  
    The path where model archive directories can be found. This path
    must be given in the model config *yaml* file. It can be given as
    an absolute path, or a relative path. If it is a relative path, 
    it should be relative to the `local_directory`.
    
These directories, especially the ones other than the `local_directory`,
are defined in a model configuration *yaml* file. This makes it easy to
change the directory pointers when moving TMIP-EMAT between different
machines that may have different file system structures.

## Single Run Operation for Development and Debugging

Before we take on the task of running this model in exploratory mode, we'll
want to make sure that our interface code is working correctly. To check each
of the components of the interface (setup, run, post-process, load-measures,
and archive), we can run each individually in sequence, and inspect the results
to make sure they are correct.

### setup

This method is the place where the core model *set up* takes place,
including creating or modifying files as necessary to prepare
for a core model run.  When running experiments, this method
is called once for each core model experiment, where each experiment
is defined by a set of particular values for both the exogenous
uncertainties and the policy levers.  These values are passed to
the experiment only here, and not in the `run` method itself.
This facilitates debugging, as the `setup` method can be used 
without the `run` method, as we do here. This allows us to manually
inspect the prepared files and ensure they are correct before
actually running a potentially expensive model.

Each input exogenous uncertainty or policy lever can potentially
be used to manipulate multiple different aspects of the underlying
core model.  For example, a policy lever that includes a number of
discrete future network "build" options might trigger the replacement
of multiple related network definition files.  Or, a single uncertainty
relating to the cost of fuel might scale both a parameter linked to
the modeled per-mile cost of operating an automobile and the
modeled total cost of fuel used by transit services.

For this demo model, running the core model itself in files mode 
requires two configuration files to be available, one for levers and
another for uncertainties.  These two files are provided in the demo
in two ways: as a runnable base file (for the levers) and as a template
file (for the uncertainties).

The levers file is a *ready-to-use* file (for this demo, in YAML format,
although your model may use a different file format for input files).
It has default values pre-coded into the file, and to modify this 
file for use by EMAT the `setup` method needs to parse and edit this
file to swap out the default values for new ones in each experiment.
This can be done using regular expressions (as in this demo), or any other method you
like to edit the file appropriately.  The advantage of this approach
is that the base file is ready to use with the core model as-is, facilitating
the use of this file outside the EMAT context.

In [5]:
#show_file_contents(fx.master_directory.name, 'road-test-files', 'demo-inputs-l.yml')

By contrast, the uncertainties file is in a *template* format. The
values of the parameters that will be manipulated by EMAT for each 
experiment are not given by default values, but instead 
each value to be set is indicated in the file by a unique token that is easy to
search and replace, and definitely not something that appear in any script otherwise.
This approach makes the text-substitution code that is used in this module much
simpler and less prone to bugs.  But there is a small downside of this approach:
every parameter must definitely be replaced in this process, as the template file
is unusable outside the EMAT context, and also every unique token needs to be replaced. 

In [6]:
#show_file_contents(fx.master_directory.name, 'road-test-files', 'demo-inputs-x.yml.template')

Regardless of which file management system you use, the `setup` method
is the place to make edits to these input files and write them into 
your working directory.  To do so,
the `setup` method takes one argument: a dictionary containing key-value
pairs that assign a particular value to each input (exogenous uncertainty 
or policy lever) that is defined in the model scope.  The keys must match 
exactly with the names of the parameters given in the scope. 

If you have written your `setup` method to call the super-class `setup`,
you will find that if you give keys as input that are not defined in
the scope, you'll get a KeyError.

In [7]:
# bad_params = {
#     'name_not_in_scope': 'is_a_problem',
# }

# try:
#     fx.setup(bad_params)
# except KeyError as error:
#     log.error(repr(error))

On the other hand, your custom model may or may not allow you to leave out
some parameters.  It is up to you to decide how to handle missing values, 
either by setting them at their default values or raising an error. In 
normal operation, parameters typically won't be left out from the design
of experiments, so it is not usually important to monitor this carefully.

In our example module's `setup`, all of the uncertainty values must be given,
because the template file would be unusable otherwise. But the policy levers 
can be omitted, and if so they are left at their default values in the 
original file.  Note that the default values in that file are not strictly
consistent with the default values in the scope file, and TMIP-EMAT does 
nothing on its own to address this discrepancy.

In [8]:
params = {
    'ValueOfTime': 13,
    'Income': 46300,
} 

fx.setup(params)

[00:03.32] MainProcess/INFO: VERSPM SETUP...
[00:03.33] MainProcess/INFO: VERSPM SETUP complete


After running `setup` successfully, we will have overwritten the 
"demo-inputs-l.yml" file with new values, and written a new 
"demo-inputs-x.yml" file into the model working directory with those
values.

In [9]:
show_dir(fx.local_directory)

tmp2v_r9ty1/
├── VERSPM/
│   ├── .Rprofile
│   ├── defs/
│   │   ├── deflators.csv
│   │   ├── geo.csv
│   │   ├── model_parameters.json
│   │   ├── run_parameters.json
│   │   └── units.csv
│   ├── inputs/
│   │   ├── azone_carsvc_characteristics.csv
│   │   ├── azone_charging_availability.csv
│   │   ├── azone_electricity_carbon_intensity.csv
│   │   ├── azone_fuel_power_cost.csv
│   │   ├── azone_gq_pop_by_age.csv
│   │   ├── azone_hh_lttrk_prop.csv
│   │   ├── azone_hh_pop_by_age.csv
│   │   ├── azone_hh_veh_mean_age.csv
│   │   ├── azone_hh_veh_own_taxes.csv
│   │   ├── azone_hhsize_targets.csv
│   │   ├── azone_lttrk_prop.csv
│   │   ├── azone_payd_insurance_prop.csv
│   │   ├── azone_per_cap_inc.csv
│   │   ├── azone_prop_sov_dvmt_diverted.csv
│   │   ├── azone_veh_use_taxes.csv
│   │   ├── azone_vehicle_access_times.csv
│   │   ├── bzone_carsvc_availability.csv
│   │   ├── bzone_dwelling_units.csv
│   │   ├── bzone_employment.csv
│   │   ├── bzone_hh_inc_qrtl_prop.csv
│   │   ├

In [10]:
show_file_contents(fx.local_directory, 'VERSPM', 'defs', 'model_parameters.json')

[{"NAME": "ValueOfTime", "VALUE": "13", "TYPE": "currency", "UNITS": "base cost year dollars per hour"}]


### run

The `run` method is the place where the core model run takes place.
Note that this method takes no arguments; all the input
exogenous uncertainties and policy levers are delivered to the
core model in the `setup` method, which will be executed prior
to calling this method. This facilitates debugging, as the `setup`
method can be used without the `run` method as we did above, allowing
us to manually inspect the prepared files and ensure they
are correct before actually running a potentially expensive model.

In [11]:
fx.run()

[00:03.35] MainProcess/INFO: VERSPM RUN ...
[02:20.27] MainProcess/INFO: VERSPM RUN complete


The `RoadTestFileModel` class includes a custom `last_run_logs` method,
which displays both the "stdout" and "stderr" logs generated by the 
model executable during the most recent call to the `run` method.
We can use this method for debugging purposes, to identify why the 
core model crashes (if it does crash).  In this first test it did not
crash, and the logs look good.

In [12]:
fx.last_run_logs()

=== STDOUT ===
run_model.R: script entered
run_model.R: library visioneval loaded
[1] "2020-07-22 16:17:57 -- Initializing Model. This may take a while."
[1] "2020-07-22 16:18:02 -- Model successfully initialized."
run_model.R: initializeModel completed
[1] "2020-07-22 16:18:02 -- Starting module 'CreateHouseholds' for year '2010'."
[1] "2020-07-22 16:18:04 -- Finish module 'CreateHouseholds' for year '2010'."
[1] "2020-07-22 16:18:04 -- Starting module 'PredictWorkers' for year '2010'."
[1] "2020-07-22 16:18:06 -- Finish module 'PredictWorkers' for year '2010'."
[1] "2020-07-22 16:18:06 -- Starting module 'AssignLifeCycle' for year '2010'."
[1] "2020-07-22 16:18:06 -- Finish module 'AssignLifeCycle' for year '2010'."
[1] "2020-07-22 16:18:06 -- Starting module 'PredictIncome' for year '2010'."
[1] "2020-07-22 16:18:09 -- Finish module 'PredictIncome' for year '2010'."
[1] "2020-07-22 16:18:09 -- Starting module 'PredictHousing' for year '2010'."
[1] "2020-07-22 16:18:12 -- Finish modu

In [14]:
show_dir(os.path.join(fx.master_directory.name, 'VERSPM', 'output'))

output/
├── Azone.csv
├── Bzone.csv
├── Household.csv
├── Marea.csv
├── Region.csv
├── Vehicle.csv
└── Worker.csv


In [ ]:
STOP

### post-process

There is an (optional) `post_process` step that is separate from the `run` step.

Post-processing differs from the main model run in two important ways:

- It can be run to efficiently generate a subset of performance measures.
- It can be run based on archived model main-run core model results.

Both features are designed to support workflows where new performance 
measures are added to the exploratory scope after the main model run(s)
are completed. By allowing the `post_process` method to be run only for a 
subset of measures, we can avoid replicating possibly expensive 
post-processing steps when we have already completed them, or when they
are not needed for a particular application.  

For example, consider an exploratory modeling activity where the scope 
at the time of the initial model run experiments was focused on highway
measures, and transit usage was not explored extensively, and no 
network assignment was done for transit trips when the experiments were
initially run.  By creating a post-process step to run the transit 
network assignment, we can apply that step to existing archived results,
as well as have it run automatically for future model experients
where transit usage is under study, but continue to omit it for future 
model experients where we do not need it.

An optional `measure_names` argument allows the post-processor to
identify which measures need additional computational effort to generate,
and to skip excluded measures that are not currently of interest, or
which have already been computed and do not need to be computed again.

The post processing is isolated from the main model run to allow it to
be run later using archived model results.  When executed directly 
after a core model run, it will operate on the results of the model
stored in the local working directory.  However, it can also be
used with an optional `output_path` argument, which can be pointed at
a model archive directory instead of the local working directory.

A consequence of this (and an intentional limitation) is that the 
`post_process` method should only use files from the set of files 
that are or will be archived from the core model run, and not attempt
to use other non-persistent temporary or intermediate files that 
will not be archived.

In [ ]:
fx.post_process()

At this point, the model's output performance measures should be available in one
or more output files that can be read in the next step.  For this example, the
results are written to two separate files: 'output_1.csv.gz' and 'output.yaml'.

In [ ]:
show_file_contents(fx.local_directory, 'road-test-files', "Outputs", "output.yaml")

Note in this example, some of the values in the `output_1.csv.gz` file
are intentionally manipulated in a contrived manner, so that there is 
some work for the post-processor to do.

In [ ]:
show_file_contents(fx.local_directory, 'road-test-files', "Outputs", "output_1.csv.gz")

### load-measures

The `load_measures` method is the place to actually reach into
files in the core model's run results and extract performance
measures, returning a dictionary of key-value pairs for the 
various performance measures. It takes an optional list giving a 
subset of performance measures to load, and like the `post_process` 
method also can be pointed at an archive location instead of loading 
measures from the local working directory (which is the default).
The `load_measures` method should not do any post-processing
of results (i.e. it should read from but not write to the model
outputs directory).

In [ ]:
fx.load_measures()

You may note that the implementation of `RoadTestFileModel` in the `core_files_demo` module
does not actually include a `load_measures` method itself, but instead inherits this method
from the `FilesCoreModel` superclass. The instructions on how to actually find the relevant
performance measures for this file are instead loaded into table parsers, which are defined
in the `RoadTestFileModel.__init__` constructor.  There are [details and illustrations
of how to write and use parsers in the file parsing examples page of the TMIP-EMAT documentation.](https://tmip-emat.github.io/source/emat.models/table_parse_example.html)

### archive

The `archive` method copies the relevant model output files to an archive location for 
longer term storage.  The particular archive location is based on the experiment id
for a particular experiment, and can be customized if desired by overloading the 
`get_experiment_archive_path` method.  This customization is not done in this demo,
so the default location is used.

In [ ]:
fx.get_experiment_archive_path(parameters=params)

Actually running the `archive` method should copy any relevant output files
from the `model_path` of the current active model into a subdirectory of `archive_path`.

In [ ]:
fx.archive(params)

In [ ]:
show_dir(fx.local_directory)

It is permissible, but not required, to simply copy the entire contents of the 
former to the latter, as is done in this example. However, if the current active model
directory has a lot of boilerplate files that don't change with the inputs, or
if it becomes full of intermediate or temporary files that definitely will never
be used to compute performance measures, it can be advisable to selectively copy
only relevant files. In that case, those files and whatever related sub-directory
tree structure exists in the current active model should be replicated within the
experiments archive directory.

## Normal Operation for Running Multiple Experiments

For this demo, we'll create a design of experiments with only 8 experiments.
The `design_experiments` method of the `RoadTestFileModel` object is not defined
in the custom `core_files_demo` written for this model, but rather is a generic
function provide by the TMIP-EMAT main library.
Real applications will typically use a larger number of experiments, but this small number
is sufficient to demonstrate the operation of the tools.

In [ ]:
design1 = fx.design_experiments(design_name='lhs_1', n_samples=8)
design1

The `run_experiments` command will automatically run the model once for each experiment in the named design.
The demo command line version of the road test model is (intentionally) a little bit slow, so will take a few
seconds to conduct these eight model experiment runs.

In [ ]:
fx.run_experiments(design_name='lhs_1')

### Re-running Failed Experiments

If you pay attention to the logged output, you might notice that one of the 
experiments (the last one) failed.  We can see `NaN` values in the outputs.

In [ ]:
results = fx.read_experiment_measures('lhs_1')
results

We can collect the id's of the failed experiments programmatically. To collect all the experiments that are
missing any performance measure output, we can do this:

In [ ]:
fails = results.isna().any(axis=1)
failed_experiment_ids = fails.index[fails]
failed_experiment_ids

When there is an error (thrown as a `subprocess.CalledProcessError`)
during the execution of a `FilesCoreModel`, the output from stdout
and stderr are written to log files in the archive location, instead
of having the legit model outputs written there.

We can see the log output by reading in the log file, like this:

In [ ]:
error_log = os.path.join(
    fx.get_experiment_archive_path(9), 
    'error.stdout.log'
)
with open(error_log, 'r') as stdout:
    error_log_content = stdout.read()
    
print(error_log_content)

Here we see the log file is explicitly taunting us about 
randomly crashing the model run.  That's fine -- we wanted to
crash the execution randomly to show what to do in this event, cause it happens 
sometimes.  Maybe a disk filled up, or there is an intermittent
license problem that causes a failure one in a while.  If that's the
case and we can fix it just by re-running, awesome!

We can load just the failed experiments to try them again.

In [ ]:
failed_experiments = fx.read_experiment_parameters(experiment_ids=failed_experiment_ids)
failed_experiments

Normally, there is a "short circuit" process that will
prevent re-running a core model experiment, instead the performance measure results
will simply be loaded from the database, which is typically much faster than
actually running the core model.  But, if the performance measures stored in the
database are junk, we will not want to trigger the short circuit system, and
actually run the full core model again.  To do so, we can disable the
short circuit like this:

In [ ]:
fx.allow_short_circuit = False

Now we can re-run the failed experiment.  If it failed because of a transient error, 
e.g. a disk space problem that's been fixed, then perhaps we can simply re-run the model
and it will work.

In [ ]:
fx.run_experiments(failed_experiments)

Much better!  Now we can see we have a more complete set of outputs, without the NaN's.  Hooray!

In [ ]:
results = fx.db.read_experiment_all(scope_name=fx.scope.name, design_name='lhs_1')
results

## Multiprocessing for Running Multiple Experiments

The examples above are all single-process demonstrations of using TMIP-EMAT to run core model
experiments.  If your core model itself is multi-threaded or otherwise is designed to make 
full use of your multi-core CPU, or if a single core model run will otherwise max out some
computational resource (e.g. RAM, disk space) then single process operation should be sufficient.

If, on the other hand, your core model is such that you can run multiple independent instances of
the model side-by-side on the same machine, then you could benefit from a multiprocessing 
approach.  This can be accomplished by splitting a design of experiments over several
processes that you start manually, or by using an automatic multiprocessing library such as 
`dask.distributed`.

### Running a Subset of Experiments Manually

Suppose, for example, you wanted to distribute the workload of running experiments over several processes,
or even over several computers. If each process has file system access to the same TMIP-EMAT database of
experiments, we can orchestrate these experiments in parallel by manually splitting up the processes.

To begin with, we'll have one process create a complete design of experiments, and save it to the 
database (which happens automatically here).

In [ ]:
design2 = fx.design_experiments(design_name='lhs_2', n_samples=8, random_seed=42)

Then, we can create set up a copy of the same model in a different process, even on a different
machine, as long as we point back to the same original database file. This implies the different
process has access to the file system where the original file is stored. It is valuable to
read and write to the same database file, not just a copy of the file, as this will obviate the need
to sync the experimental data manually afterwards.  In this demo, we'll 
just create a new directory to work in, but we'll point to the database in the original directory.
Instead of allowing our model to implicitly create a new database file in the new directory, we'll
instantiate a SQLiteDB object pointing to the original database.

In [ ]:
database_filename = fx.db.database_path
db2 = emat.SQLiteDB(database_filename)

Now, `db2` is a `emat.SQLiteDB` object, which wraps a *new* connection to the *original* database.
Then, we'll pass that `db2` explicitly to the new `RoadTestFileModel` constructor, which will
create a complete copy of our model (other than the database) in a new directory.

In [ ]:
fx2 = core_files_demo.RoadTestFileModel(db=db2)

To run a particular slice of a design of experiments, we need to load the experimental design first, 
and then pass that slice to the `run_experiments` function, instead of just giving the `design_name`.

In [ ]:
design2 = fx.read_experiment_parameters('lhs_2')

For splitting the work across a number of similarly capable processes or machines,
the double-colon slice is convenient.  If, for example, you are splitting the work
over 4 computers, you can run each with slices `0::4`, `1::4`, `2::4`, and `3::4`.
This slices in skip-step manner, so slice below will run every 4th experiment
from the design, starting with experiment index 0 (i.e. the first one).  

In [ ]:
fx2.run_experiments(design2.iloc[0::4])

Because we have linked the second model instance back to the same database, after
these experiments have finished we can access the results from the original `fx`
instance.

In [ ]:
fx.read_experiment_measures('lhs_2')

It is important to note that for this manual multiprocessing technique to work, where
different processes run the model simultaneously, each process must be in a seperate 
Python instance (e.g. in seperate Jupyter notebooks, not in the same notebook as shown here).

### Automatic Multiprocessing for Running Multiple Experiments

The examples above are all essentially single-process demonstrations of using TMIP-EMAT to run core model
experiments, either by running all in one single process, or by having a user manually instantiate a number 
of single processes.  If your core model itself is multi-threaded or otherwise is designed to make 
full use of your multi-core CPU, or if a single core model run will otherwise max out some
computational resource (e.g. RAM, disk space) then single process operation should be sufficient.

If, on the other hand, your model is such that you can run multiple independent instances of
the model side-by-side on the same machine, but you don't want to manage the process of manually, 
then you could benefit from a multiprocessing approach that uses the `dask.distributed` library.  To
demonstrate this, we'll create yet another small design of experiments to run.

In [ ]:
design3 = fx.design_experiments(design_name='lhs_3', n_samples=8, random_seed=3)
design3

The demo module is set up to facilitate distributed multiprocessing. During the `setup`
step, the code detects if it is being run in a distributed "worker" environment instead of
in a normal Python environment.  If the "worker" environment is detected, then a copy
of the entire files-based model is made into the worker's local workspace, and the model
is run there instead of in the master workspace.  This allows each worker to edit the files
independently and simultaneously, without disturbing other parallel workers.

With this small modification, we are ready to run this demo model in parallel subprocesses.
to do, we simply import the `get_client` function, and use that for the `evaluator` argument
in the `run_experiments` method.

In [ ]:
from emat.util.distributed import get_client # for multi-process operation
fx.run_experiments(design=design3, evaluator=get_client())

## Running without a Database

It is possible to use the methods from TMIP-EMAT without connecting to a 
database at all, although this is not recommended.  The demo `RoadTestFileModel`
by default creates a database for you if one is not given, but it also 
allows explicitly disclaiming a database by setting the `db` argument to `False`.

In [ ]:
fx_nodb = core_files_demo.RoadTestFileModel(db=False)

Without a database, the methods still work, but the data is not stored anywhere persistent.
So, for example, the `design_experiments` will return a pandas DataFrame containing a 
design, but not store it.

In [ ]:
design4 = fx_nodb.design_experiments(design_name='lhs_4', n_samples=8, random_seed=4)
design4

If you try to run these experiments by giving the design name, it will fail,
because there is no database to query to convert that name into experimental
parameters.

In [ ]:
try:
    fx_nodb.run_experiments('lhs_4')
except ValueError as error:
    log.error(repr(error))

In [ ]:
from emat.examples import road_test

In [ ]:
_s, _db, _m = road_test()

In [ ]:
_m.run_experiments(design3, db=False)